---
# Caso Mail Spam

Los modelos bayesianos, en particular el clasificador bayesiano ingenuo (Naive Bayes), son ampliamente utilizados en la detección de spam debido a varias razones:

- **Eficiencia computacional**: Los modelos bayesianos son computacionalmente eficientes y pueden manejar grandes volúmenes de datos de manera eficaz, lo que los hace adecuados para aplicaciones en las que se procesan grandes cantidades de correos electrónicos.

- **Tratamiento de datos textuales**: El clasificador bayesiano ingenuo es especialmente adecuado para el procesamiento de datos textuales, como los contenidos de los correos electrónicos. Puede manejar características categóricas o discretas, lo que lo hace efectivo para analizar palabras clave y patrones de texto en los correos electrónicos.

- **Capacidad para modelar probabilidades condicionales**: El enfoque bayesiano permite modelar probabilidades condicionales de que un correo electrónico sea spam o no spam dadas las características observadas en el correo electrónico. Esto se alinea bien con la naturaleza de la detección de spam, que implica hacer predicciones basadas en la probabilidad de que un correo electrónico sea spam dado su contenido.

- **Adaptabilidad y actualización continua**: Los modelos bayesianos pueden adaptarse y actualizarse fácilmente con nuevos datos, lo que los hace adecuados para la detección de spam en entornos dinámicos donde los patrones de spam pueden cambiar con el tiempo


En este caso, entrenaremos un algoritmo de clasificación de correos spam a partir del dataset https://archive.ics.uci.edu/ml/datasets/spambase con el clasificador Bayesiano.

### Importación de librerías

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Lectura de los datos

In [3]:
# En el archivo spambase.names vienen los nombres de las 56 columnas
df_names = pd.read_csv('spambase.names', sep=':', header=None)

In [4]:
df_names.shape

(58, 2)

In [8]:
df_names[0]

0                 word_freq_make
1              word_freq_address
2                  word_freq_all
3                   word_freq_3d
4                  word_freq_our
5                 word_freq_over
6               word_freq_remove
7             word_freq_internet
8                word_freq_order
9                 word_freq_mail
10             word_freq_receive
11                word_freq_will
12              word_freq_people
13              word_freq_report
14           word_freq_addresses
15                word_freq_free
16            word_freq_business
17               word_freq_email
18                 word_freq_you
19              word_freq_credit
20                word_freq_your
21                word_freq_font
22                 word_freq_000
23               word_freq_money
24                  word_freq_hp
25                 word_freq_hpl
26              word_freq_george
27                 word_freq_650
28                 word_freq_lab
29                word_freq_labs
30        

In [10]:
# en el archivo spambase.data viene la data de las 56 columnas
df = pd.read_csv('spambase.data', names=df_names[0])


In [ ]:
# verifique los primeros registros del dataframe
df.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.0,0.00,...,0.0,0.000,0.0,0.778,0.00,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.0,0.94,...,0.0,0.132,0.0,0.372,0.18,0.048,5.114,101,1028,1


In [ ]:
# imprima las dimensiones del dataframe
df.shape

### Limpieza de datos

In [ ]:
# verifique la estructura del set de datos
df.info()

In [ ]:
# verifique si tiene valore nulos, de ser así, haga un tratamiento de ellos
df.isnull().sum()

### Definición del modelo

Defina un modelo con todas los features. Note que la última columna (spam) contiene la variable objetivo (es spam o no es spam).

In [ ]:
# Todas las columnas excepto 'spam' serán features
X = df.drop('spam', axis=1)

// La columna 'spam' es la variable objetivo (0 = no spam, 1 = spam)
y = df['spam']

In [ ]:
# verifique los valores únicos de la variable objetivo
print("Conteo de clases:")
print(y.value_counts())
print("\nProporción de clases:")
print(y.value_counts(normalize=True))

### Escalamiento de los datos

No, generalmente no es necesario escalar los datos cuando se utiliza el clasificador bayesiano ingenuo (Naive Bayes). Esto se debe a que el Naive Bayes no hace suposiciones sobre la distribución de las características y trata cada característica de manera independiente, asumiendo que son condicionalmente independientes dado el valor de la clase objetivo.

Dado que el Naive Bayes calcula las probabilidades condicionales de cada característica dada la clase objetivo, el escalado de las características no afecta directamente estas probabilidades condicionales. Por lo tanto, no se requiere escalado para el clasificador Naive Bayes.

Sin embargo, existen algunas excepciones en las que el escalado de características puede ser útil, como en el caso de características que tienen escalas muy diferentes o en conjuntos de datos donde las características están altamente sesgadas. En tales casos, el escalado puede ayudar a mejorar el rendimiento del modelo.

En resumen, mientras que el escalado de características puede ser beneficioso en otros algoritmos de machine learning, como los basados en distancias (por ejemplo, k-Nearest Neighbors) o en gradientes (por ejemplo, Support Vector Machines), generalmente no es necesario para el clasificador bayesiano ingenuo debido a su naturaleza y suposiciones específicas.

In [ ]:
# import
from sklearn.preprocessing import StandardScaler

En esta guía compararemos el performance del modelo bayesiano con datos sin escalar y frente a un modelo con los datos escalados. Por lo tanto, dejaremos los datos escalados en una variable distinta para su utilización posterior.

In [ ]:
# escalar el set de datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled[:5]

### Validación Cruzada (Datos sin Escalar)

Realizaremos la división del set de test y de entrenamiento con los datos sin escalar.


In [ ]:
# import
from sklearn.model_selection import train_test_split

In [ ]:
# dividir set de entrenamiento y test (datos sin escalar)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
X_train.shape, X_test.shape

### Entrenamiento Algoritmo


El algoritmo de Naive Bayes tiene varias ventajas que lo hacen ampliamente utilizado en problemas de clasificación. Aquí están algunas de las principales ventajas:

- **Simplicidad y facilidad de implementación**: Naive Bayes es un algoritmo simple y fácil de entender. Su naturaleza intuitiva y sus suposiciones simples hacen que sea fácil de implementar y comprender, lo que lo hace adecuado para aplicaciones prácticas.

- **Eficiencia computacional**: Naive Bayes es computacionalmente eficiente y puede manejar grandes volúmenes de datos de manera efectiva. Los tiempos de entrenamiento y predicción suelen ser rápidos, lo que lo hace adecuado para aplicaciones en tiempo real o con grandes conjuntos de datos.

- **Bajo riesgo de sobreajuste**: Debido a su simplicidad y sus suposiciones de independencia condicional, Naive Bayes tiende a tener un bajo riesgo de sobreajuste, especialmente en conjuntos de datos con muchas características. Esto lo hace robusto y generalizable a nuevos datos.

- **Manejo eficaz de datos categóricos y textuales**: Naive Bayes es especialmente adecuado para problemas con características categóricas o textuales, como la clasificación de documentos de texto o la detección de spam de correo electrónico. Puede manejar estas características de manera efectiva sin necesidad de preprocesamiento complejo.

- **Capacidad para modelar probabilidades**: Naive Bayes proporciona una forma natural de modelar probabilidades condicionales y de hacer predicciones basadas en estas probabilidades. Esto lo hace útil para problemas de clasificación donde se requiere una estimación de la probabilidad de pertenencia a una clase.

- **Adaptabilidad a conjuntos de datos desequilibrados**: Naive Bayes puede funcionar bien incluso en conjuntos de datos desequilibrados, donde hay una gran diferencia en la cantidad de muestras por clase. Esto se debe a que el algoritmo se basa en la probabilidad y no en la distribución de las muestras.




In [58]:
# Importar algoritmo bayesiano
from sklearn.naive_bayes import GaussianNB

In [ ]:
# Ajustar modelo Naive Bayes con datos sin escalar
model = GaussianNB()
model.fit(X_train, y_train)

In [ ]:
# Accuracy simple en entrenamiento y test
print(f"Accuracy en entrenamiento: {model.score(X_train, y_train):.3f}")
print(f"Accuracy en test        : {model.score(X_test, y_test):.3f}")

### Métricas de Evaluación

Al evaluar las métricas de clasificación de un algoritmo de detección de spam, es importante tener en cuenta varios aspectos para asegurar una evaluación precisa y significativa del rendimiento del modelo. Algunos de los cuidados que debemos tener incluyen:

- **Desbalance de clases**: En problemas de detección de spam, es común que las clases estén desbalanceadas, es decir, que haya muchas más instancias de correos legítimos que de spam. Esto puede sesgar las métricas de evaluación, especialmente si el algoritmo predice la clase dominante en la mayoría de los casos. Es importante considerar métricas que sean robustas frente al desbalance de clases, como precision, recall y F1-score.

- **Evaluación en datos no vistos**: Es fundamental evaluar el rendimiento del modelo en un conjunto de datos no vistos o conjunto de prueba independiente (set de test). Esto garantiza que el modelo sea capaz de generalizar a nuevos correos electrónicos y no solo memorice los datos de entrenamiento.

- **Métricas apropiadas**: Las métricas de evaluación deben seleccionarse cuidadosamente según las necesidades y objetivos del problema. Por ejemplo, precision y recall son útiles para evaluar el equilibrio entre la capacidad de identificar spam (recall) y la precisión de estas predicciones (precision).

- **Considerar costos asociados**: Dependiendo del contexto, los costos asociados con los errores de clasificación pueden variar. Por ejemplo, el costo de clasificar incorrectamente un correo legítimo como spam puede ser diferente del costo de clasificar incorrectamente un correo de spam como legítimo. Es importante tener en cuenta estos costos al interpretar las métricas de evaluación y al tomar decisiones sobre la configuración del modelo.

- **Interpretación de las métricas**: Es esencial comprender el significado de las métricas de evaluación y cómo se relacionan con el problema específico de detección de spam. Por ejemplo, un alto recall indica que el modelo es capaz de detectar la mayoría de los correos de spam, mientras que un alto precision indica que las predicciones positivas del modelo son en su mayoría correctas.

In [ ]:
# importar librerias de métricas
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

In [ ]:
# realizar predicciones sobre el X_test (datos sin escalar)
y_pred = model.predict(X_test)
y_pred[:10]

In [ ]:
# Calcular métricas de evaluación para datos sin escalar
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f"Accuracy : {acc:.3f}")
print(f"Precision: {prec:.3f}")
print(f"Recall   : {rec:.3f}")
print(f"F1-score : {f1:.3f}")
print("\nReporte de clasificación (Naive Bayes sin escalar):")
print(classification_report(y_test, y_pred))

In [ ]:
# Matriz de confusión para modelo Naive Bayes (sin escalar)
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap='Blues')
plt.title('Matriz de confusión - Naive Bayes (sin escalar)')
plt.show()

In [ ]:
# Tabla de contingencia Real vs Predicción (sin escalar)
pd.crosstab(y_test, y_pred, rownames=['Real'], colnames=['Predicción'])

### Comparación con Datos Escalados

En esta sección, realizaremos el entrenamiento con los datos escalados para ver si hay diferencias en el performance del modelo.


In [ ]:
# validación cruzada con datos escalados
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42, stratify=y
)
X_train_s.shape, X_test_s.shape

In [ ]:
# Entrenamiento con datos escalados
model_scaled = GaussianNB()
model_scaled.fit(X_train_s, y_train_s)

In [ ]:
# Métricas de evaluación con datos escalados
y_pred_s = model_scaled.predict(X_test_s)
acc_s = accuracy_score(y_test_s, y_pred_s)
prec_s = precision_score(y_test_s, y_pred_s)
rec_s = recall_score(y_test_s, y_pred_s)
f1_s = f1_score(y_test_s, y_pred_s)
print(f"Accuracy (scaled) : {acc_s:.3f}")
print(f"Precision (scaled): {prec_s:.3f}")
print(f"Recall (scaled)   : {rec_s:.3f}")
print(f"F1-score (scaled) : {f1_s:.3f}")

### Conclusiones y Reflexiones

Por ultimo realizaremos un entrenamiento del modelo con el valor de K determinado anteriormente. Algunas preguntas que pueden ayudar a reflexionar:

- En este problema, ¿mejora el performance respecto a otros algoritmos utilizadas previamente? 
- En este caso, ¿se produce una mejora con el escalamiento?
- ¿Cuándo sería recomendable utilizar este tipo de algoritmo?

In [ ]:
# Conclusiones aquí
"""
- Resume las diferencias de desempeño entre Naive Bayes sin escalar y con datos escalados.
- Comenta si el escalamiento ayudó o no al modelo bayesiano en este caso.
- Compara (si lo recuerdas) con otros modelos vistos (por ejemplo KNN, Regresión Logística),
  pensando en ventajas/desventajas de Naive Bayes para el problema de spam.
"""

Si llegaste hasta acá Eres un crack!

---